<a href="https://colab.research.google.com/github/Sulaiman29/IntelliBrief/blob/main/BDA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fasttext

In [ ]:
pip install rouge-score

In [ ]:
import fasttext
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
from scipy import spatial
from statistics import mean
from math import ceil
from rouge_score import rouge_scorer
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/

In [ ]:
model_skipgram = fasttext.load_model("models/model_skipgram.bin")
model_cbow = fasttext.load_model("models/model_cbow.bin")

In [ ]:
from IPython.display import Image

# Provide the path to your PNG file
image_path = '/content/473172_1_En_16_Fig1_HTML.png'

# Display the image
Image(filename=image_path)


In [ ]:
!unzip -q /content/NLP.zip -d /content/data

In [ ]:
#Naive baye's classifier:
import pandas as pd #Dataframe Manipulation library
import numpy as np #Data Manipulation library

#sklearn modules for Feature Extraction & Modelling
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#Libraries for Plotting
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from pylab import boxplot, text
import seaborn as sns
import joblib
import os
import glob

In [ ]:
print(f"Current working directory is: {os.getcwd()}")
dir = os.chdir("/content")
print(f'Changing directory to {os.getcwd()}')

In [ ]:
def load_data(folder_names, root_path):
    fileNames = [path + "/data/BBC News Summary/News Articles/" + folder + '/' + "*.txt"
        for path,folder in zip([root_path]*len(folder_names), folder_names)]
    #print(fileNames)
    #print("\n")
    doc_list = []
    tags = folder_names
    for docs in fileNames:
        #print(docs)
        doc = glob.glob(docs)#glob method iterates through all files and reads the text in documents in the folders
        for text in doc:
            with open(text, encoding="latin-1") as f:
                topic = docs.split('/')[len(docs.split('/'))-2]
                lines = f.readlines()
                heading = lines[0].strip()#stripping the text by spaces and using first element into heading
                body = ' '.join([l.strip() for l in lines[1:]])
                doc_list.append([topic,heading,body])
        print(f"Loading data from \033[1m{topic}\033[0m directory")
    print("\nEntire Data is loaded successfully")

    return doc_list

In [ ]:
folder_names = ['business','entertainment','politics','sport','tech']
docs = load_data(folder_names=folder_names,root_path=os.getcwd())

In [ ]:
docs = pd.DataFrame(docs, columns = ['Category','Heading','Article'])

In [ ]:
docs.head()

In [ ]:
docs['text_len']=docs.Heading.apply(len)
docs['headlines_len']=docs.Article.apply(len)

In [ ]:
docs['Category'].value_counts().plot.bar()